In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 523cbb7
molecule = {'n2o': 6.4e-07}
band = [9]
atmpro = mls
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o n2o band7
  atmpro = mls
  band = 7
  commitnumber = a06b618
  conc = 3.2e-07
  dv = 0.001
  klin = 2.22e-20
  molecule = n2o
  ng_adju = [0, 0]
  ng_refs = [2, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 1380
  vmin = 1215
  w_diffuse = [(1.8, 1.66), (1.6, 1.8)]
  wgt = [(0.6, 0.5), (0.6, 0.9)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.066466                    1 -0.037866
0.000750        2 -0.066013                    2 -0.037577
0.001052        3 -0.064951                    3 -0.036929
0.001476        4 -0.063824                    4 -0.036233
0.002070        5 -0.062750                    5 -0.035553
0.002904        6 -0.061694                    6 -0.034836
0.004074        7 -0.060690                    7 -0.034136
0.005714        8 -0.059759                    8 -0.033397
0.008015        9 -0.058955                    9 -0.032682
0.011243       10 -0.058304                   10 -0.031930
0.015771       11 -0.057884                   11 -0.031183
0.022122       12 -0.057786                   12 -0.030453
0.031031       13 -0.058155                   13 -0.029720
0.043528       14 -0.059099                   14 -0.028979
0.061057       15 -0.057850                   15 -0.026434
0.085645       16 -0.048607                   16 -0.018535
0.120136       17 -0.031317                   17 -0.004976
0.168516       18 -0.010164                   18  0.012298
0.236378       19  0.014922                   19  0.033642
0.331549       20  0.044621                   20  0.059812
0.465100       21  0.079993                   21  0.091736
0.652400       22  0.123086                   22  0.130081
0.915100       23  0.161698                   23  0.165058
1.283650       24  0.153555                   24  0.165784
1.800600       25  0.107567                   25  0.133562
2.525700       26  0.068021                   26  0.097000
3.542800       27  0.042497                   27  0.064810
4.969550       28  0.027628                   28  0.039314
6.970850       29  0.019634                   29  0.021618
9.778100       30  0.014829                   30  0.010794
13.715850      31  0.010590                   31  0.004595
19.239350      32  0.006256                   32  0.001051
26.987250      33  0.002902                   33 -0.000876
37.855300      34  0.000302                   34 -0.002938
53.100050      35 -0.003164                   35 -0.007125
73.887500      36 -0.007736                   36 -0.013490
97.662500      37 -0.011874                   37 -0.019784
121.437500     38 -0.015457                   38 -0.025341
145.212500     39 -0.020072                   39 -0.031171
168.987500     40 -0.024977                   40 -0.036435
192.762500     41 -0.027214                   41 -0.038910
216.537500     42 -0.026338                   42 -0.038468
240.312500     43 -0.024950                   43 -0.037041
264.087500     44 -0.023509                   44 -0.035246
287.862500     45 -0.021906                   45 -0.033101
311.637500     46 -0.020096                   46 -0.030636
335.412500     47 -0.018067                   47 -0.027825
359.187500     48 -0.015820                   48 -0.024765
382.962500     49 -0.013372                   49 -0.021450
406.737500     50 -0.010752                   50 -0.017931
430.512500     51 -0.008026                   51 -0.014285
454.287500     52 -0.005268                   52 -0.010602
478.062500     53 -0.002523                   53 -0.006857
501.837500     54  0.000217                   54 -0.003119
525.612500     55  0.002991                   55  0.000656
549.387500     56  0.005841                   56  0.004507
573.162500     57  0.008796                   57  0.008456
596.937500     58  0.011873                   58  0.012521
620.712500     59  0.015086                   59  0.016711
644.487500     60  0.018448                   60  0.021024
668.262500     61  0.021973                   61  0.025465
692.037500     62  0.025681                   62  0.030015
715.812500     63  0.029597                   63  0.034770
739.587500     64  0.033739                   64  0.039663
763.362500     65  0.038112                   65  0.044662
78

# Fluxes by Layer

CRD                          CLIRAD                \
                        flug      fldg      fnetg       flug          fldg   
pressure    level                                                            
0.000000    1     -19.692053  0.000000 -19.692053 -19.170454  8.919774e-08   
0.000624    2     -19.692060  0.000002 -19.692058 -19.170458  1.191320e-06   
0.000876    3     -19.692063  0.000003 -19.692060 -19.170460  1.640976e-06   
0.001229    4     -19.692067  0.000004 -19.692063 -19.170462  2.285623e-06   
0.001723    5     -19.692072  0.000005 -19.692067 -19.170465  3.210406e-06   
0.002417    6     -19.692080  0.000008 -19.692072 -19.170469  4.535732e-06   
0.003391    7     -19.692090  0.000011 -19.692079 -19.170475  6.436978e-06   
0.004757    8     -19.692104  0.000016 -19.692089 -19.170484  9.162222e-06   
0.006672    9     -19.692125  0.000022 -19.692102 -19.170495  1.307149e-05   
0.009359    10    -19.692153  0.000032 -19.692121 -19.170511  1.867374e-05   
0.013128    11    -19.692193  0.000046 -19.692147 -19.170533  2.670853e-05   
0.018415    12    -19.692248  0.000065 -19.692183 -19.170564  3.822566e-05   
0.025830    13    -19.692326  0.000092 -19.692234 -19.170608  5.472225e-05   
0.036232    14    -19.692436  0.000130 -19.692306 -19.170668  7.835299e-05   
0.050823    15    -19.692592  0.000184 -19.692408 -19.170752  1.122166e-04   
0.071291    16    -19.692811  0.000262 -19.692548 -19.170867  1.630034e-04   
0.100000    17    -19.693105  0.000391 -19.692714 -19.171015  2.482070e-04   
0.140271    18    -19.693481  0.000618 -19.692863 -19.171192  4.009958e-04   
0.196760    19    -19.693942  0.001011 -19.692931 -19.171382  6.741976e-04   
0.275997    20    -19.694472  0.001681 -19.692791 -19.171551  1.158109e-03   
0.387100    21    -19.695007  0.002803 -19.692204 -19.171612  2.006371e-03   
0.543100    22    -19.695375  0.004650 -19.690725 -19.171390  3.480512e-03   
0.761700    23    -19.695166  0.007628 -19.687538 -19.170545  6.003952e-03   
1.068500    24    -19.693738  0.012076 -19.681662 -19.168615  1.007299e-02   
1.498800    25    -19.691130  0.017295 -19.673835 -19.165644  1.555321e-02   
2.102400    26    -19.688104  0.021961 -19.666143 -19.162196  2.165597e-02   
2.949000    27    -19.685001  0.025679 -19.659322 -19.158562  2.775139e-02   
4.136600    28    -19.681972  0.028628 -19.653343 -19.154923  3.323087e-02   
5.802500    29    -19.679158  0.031267 -19.647891 -19.151452  3.751867e-02   
8.139200    30    -19.676655  0.034198 -19.642457 -19.148303  4.035373e-02   
11.417000   31    -19.674619  0.037920 -19.636699 -19.145737  4.197910e-02   
16.014700   32    -19.673589  0.042658 -19.630931 -19.144485  4.323005e-02   
22.464000   33    -19.674744  0.048592 -19.626152 -19.145989  4.553669e-02   
31.510500   34    -19.679555  0.056513 -19.623042 -19.152374  5.098276e-02   
44.200100   35    -19.690118  0.067530 -19.622588 -19.167222  6.141365e-02   
62.000000   36    -19.711405  0.082146 -19.629259 -19.198234  7.740114e-02   
85.775000   37    -19.751400  0.100354 -19.651045 -19.256893  9.806528e-02   
109.550000  38    -19.803298  0.118811 -19.684488 -19.332960  1.184088e-01   
133.325000  39    -19.866503  0.138485 -19.728018 -19.425281  1.393549e-01   
157.100000  40    -19.943088  0.158542 -19.784546 -19.534432  1.607110e-01   
180.875000  41    -20.035146  0.180256 -19.854891 -19.659937  1.835923e-01   
204.650000  42    -20.140247  0.208713 -19.931535 -19.797926  2.119860e-01   
228.425000  43    -20.253616  0.247904 -20.005712 -19.943918  2.496288e-01   
252.200000  44    -20.373278  0.297300 -20.075978 -20.095772  2.971522e-01   
275.975000  45    -20.498309  0.356123 -20.142186 -20.252319  3.544238e-01   
299.750000  46    -20.627917  0.424036 -20.203880 -20.412518  4.213914e-01   
323.525000  47    -20.761341  0.500864 -20.260477 -20.575379  4.979618e-01   
347.300000  48    -20.897842  0.586484 -20.311358 -20.739985  5.841947e-01   
371.075000  49    -21.036694  0.680780 -20.3559

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')